In [99]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as geom
import fiona
import pyproj

Import Top Fast Food Inspections

In [100]:
df_FastFood = pd.read_csv('TopFastFood_Inspections.csv')
df_FastFood.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,10/27/2016,Violations were cited in the following area(s).,...,05/16/2020,Cycle Inspection / Re-inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60
1,40363945,DOMINO'S,Manhattan,148,WEST 72 STREET,10023.0,2125010200,Pizza,08/20/2019,Violations were cited in the following area(s).,...,05/16/2020,Cycle Inspection / Re-inspection,40.778074,-73.980401,107.0,6.0,15700.0,1081483.0,1.011430e+09,MN14
2,40370167,BURGER KING,Queens,22210,NORTHERN BOULEVARD,11361.0,7182251695,Hamburgers,10/31/2019,Violations were cited in the following area(s).,...,05/16/2020,Cycle Inspection / Re-inspection,40.761859,-73.759199,411.0,19.0,147100.0,4160989.0,4.074740e+09,QN46
3,40370238,BURGER KING,Queens,6815,NORTHERN BOULEVARD,11377.0,7187794875,Hamburgers,03/28/2018,Violations were cited in the following area(s).,...,05/16/2020,Cycle Inspection / Initial Inspection,40.754252,-73.897818,403.0,25.0,30902.0,4026245.0,4.011640e+09,QN28
4,40370916,BURGER KING,Queens,25810,HILLSIDE AVENUE,11004.0,7183476880,Hamburgers,09/21/2017,Violations were cited in the following area(s).,...,05/16/2020,Cycle Inspection / Initial Inspection,40.736974,-73.709922,413.0,23.0,157903.0,4179815.0,4.087880e+09,QN44


In [101]:
df_FastFood.shape

(435, 26)

In [102]:
df_FastFood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 26 columns):
CAMIS                    435 non-null int64
DBA                      435 non-null object
BORO                     435 non-null object
BUILDING                 435 non-null object
STREET                   435 non-null object
ZIPCODE                  427 non-null float64
PHONE                    435 non-null object
CUISINE DESCRIPTION      435 non-null object
INSPECTION DATE          435 non-null object
ACTION                   422 non-null object
VIOLATION CODE           417 non-null object
VIOLATION DESCRIPTION    416 non-null object
CRITICAL FLAG            416 non-null object
SCORE                    402 non-null float64
GRADE                    365 non-null object
GRADE DATE               363 non-null object
RECORD DATE              435 non-null object
INSPECTION TYPE          422 non-null object
Latitude                 435 non-null float64
Longitude                435 non-null 

Convert fast food's lat, long locations as points and put into a geopandas dataframe

In [103]:
# convert longitude, latitude to xy Points
geom = [geom.Point(xy) for xy in zip(df_FastFood['Longitude'],
                                     df_FastFood['Latitude'])]

df_FastFood = df_FastFood.drop(['Longitude', 'Latitude'], axis = 1)

crs = {'init': 'epsg:4326'}

gdf_FastFood = gpd.GeoDataFrame(df_FastFood, crs = crs, geometry = geom)

In [104]:
df_FastFood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
CAMIS                    435 non-null int64
DBA                      435 non-null object
BORO                     435 non-null object
BUILDING                 435 non-null object
STREET                   435 non-null object
ZIPCODE                  427 non-null float64
PHONE                    435 non-null object
CUISINE DESCRIPTION      435 non-null object
INSPECTION DATE          435 non-null object
ACTION                   422 non-null object
VIOLATION CODE           417 non-null object
VIOLATION DESCRIPTION    416 non-null object
CRITICAL FLAG            416 non-null object
SCORE                    402 non-null float64
GRADE                    365 non-null object
GRADE DATE               363 non-null object
RECORD DATE              435 non-null object
INSPECTION TYPE          422 non-null object
Community Board          427 non-null float64
Council District         427 non-null 

In [105]:
gdf_FastFood['geometry']

0          POINT (-73.962081374927 40.662651518808)
1          POINT (-73.980401099931 40.778073572727)
2          POINT (-73.7591993411 40.76185911524099)
3        POINT (-73.89781764958001 40.754251920542)
4         POINT (-73.70992194527899 40.73697353398)
5        POINT (-73.91980597180101 40.652872037851)
6        POINT (-73.86656564973001 40.731773141489)
7        POINT (-73.78098427964301 40.667935318774)
8        POINT (-73.85314065129799 40.834031846299)
9          POINT (-73.951791635813 40.810436705074)
10         POINT (-73.736371492069 40.718059966777)
11          POINT (-74.007362516356 40.71563717522)
12         POINT (-73.994655319797 40.743669576786)
13       POINT (-74.13075747230199 40.612913560092)
14         POINT (-73.941831944483 40.822926167108)
15          POINT (-73.88424026446 40.749416478493)
16       POINT (-73.96532040323002 40.755033261103)
17       POINT (-73.81390306639301 40.702539311298)
18         POINT (-73.744160354908 40.730135418382)
19       POI

Export the fast food geojson points to a geojson file

In [107]:
gdf_FastFood.to_file('nyc_FastFood.geojson', driver = 'GeoJSON', encoding = 'utf-8' )

#### Aggregate by Community District

In [109]:
df_cd = gpd.read_file('Population_Obesity_CD.geojson').to_crs(fiona.crs.from_epsg(4326))
df_cd

,boro_cd,shape_area,shape_leng,GEO_ID,GEO_LABEL,GEO_DISPLAY_NAME,total_population_cd,obesity_cd,geometry
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...
1,102.0,3.768921e+07,34130.595861,102.0,Manhattan CD 002,Greenwich Village and Soho,101000.735022,26,"POLYGON ((-73.9968399392649 40.7373608893982, ..."
2,103.0,4.687970e+07,30468.340600,103.0,Manhattan CD 003,Lower East Side and Chinatown,103831.154791,32,"POLYGON ((-73.98877931684083 40.7339653992359,..."
3,104.0,4.931179e+07,67623.946684,104.0,Manhattan CD 004,Clinton and Chelsea,126139.483095,24,POLYGON ((-73.99393694248319 40.77317951096692...
4,105.0,4.379030e+07,35288.305200,105.0,Manhattan CD 005,Midtown,193083.299602,21,POLYGON ((-73.97301487176121 40.76427887944521...
5,106.0,3.872909e+07,42705.937484,106.0,Manhattan CD 006,Stuyvesant Town and Turtle Bay,193731.204933,15,(POLYGON ((-73.96128450788402 40.7301590617016...
6,107.0,5.315282e+07,39863.701384,107.0,Manhattan CD 007,Upper West Side,103174.978364,28,POLYGON ((-73.95964685423152 40.80115642325645...
7,108.0,5.516880e+07,53561.096900,108.0,Manhattan CD 008,Upper East Side,164500.003324,28,(POLYGON ((-73.94180032729426 40.7690469266246...
8,109.0,4.189218e+07,34959.184938,109.0,Manhattan CD 009,Morningside Heights and Hamilton Heights,166931.655361,26,POLYGON ((-73.94014131210091 40.83037198888071...
9,110.0,3.908463e+07,35825.290700,110.0,Manhattan CD 010,Central Harlem,88571.946738,41,POLYGON ((-73.93445239397371 40.83598096014986...


In [110]:
df_cd.shape

(59, 9)

In [111]:
df_FastFood = gpd.read_file('nyc_FastFood.geojson').to_crs(fiona.crs.from_epsg(4326))
df_FastFood

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,GRADE DATE,RECORD DATE,INSPECTION TYPE,Community Board,Council District,Census Tract,BIN,BBL,NTA,geometry
0,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,10/27/2016,Violations were cited in the following area(s).,...,10/27/2016,05/16/2020,Cycle Inspection / Re-inspection,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,POINT (-73.962081374927 40.662651518808)
1,40363945,DOMINO'S,Manhattan,148,WEST 72 STREET,10023.0,2125010200,Pizza,08/20/2019,Violations were cited in the following area(s).,...,08/20/2019,05/16/2020,Cycle Inspection / Re-inspection,107.0,6.0,15700.0,1081483.0,1.011430e+09,MN14,POINT (-73.980401099931 40.778073572727)
2,40370167,BURGER KING,Queens,22210,NORTHERN BOULEVARD,11361.0,7182251695,Hamburgers,10/31/2019,Violations were cited in the following area(s).,...,10/31/2019,05/16/2020,Cycle Inspection / Re-inspection,411.0,19.0,147100.0,4160989.0,4.074740e+09,QN46,POINT (-73.7591993411 40.76185911524099)
3,40370238,BURGER KING,Queens,6815,NORTHERN BOULEVARD,11377.0,7187794875,Hamburgers,03/28/2018,Violations were cited in the following area(s).,...,03/28/2018,05/16/2020,Cycle Inspection / Initial Inspection,403.0,25.0,30902.0,4026245.0,4.011640e+09,QN28,POINT (-73.89781764958001 40.754251920542)
4,40370916,BURGER KING,Queens,25810,HILLSIDE AVENUE,11004.0,7183476880,Hamburgers,09/21/2017,Violations were cited in the following area(s).,...,09/21/2017,05/16/2020,Cycle Inspection / Initial Inspection,413.0,23.0,157903.0,4179815.0,4.087880e+09,QN44,POINT (-73.70992194527899 40.73697353398)
5,40370917,BURGER KING,Brooklyn,9006,CHURCH AVENUE,11236.0,7184985574,Hamburgers,12/26/2019,Violations were cited in the following area(s).,...,12/26/2019,05/16/2020,Cycle Inspection / Initial Inspection,317.0,42.0,88800.0,3103358.0,3.047110e+09,BK96,POINT (-73.91980597180101 40.652872037851)
6,40372422,BURGER KING,Queens,9283,QUEENS BOULEVARD,11374.0,7185750116,Hamburgers,02/08/2019,Violations were cited in the following area(s).,...,02/08/2019,05/16/2020,Cycle Inspection / Initial Inspection,406.0,29.0,71702.0,4050127.0,4.020750e+09,QN18,POINT (-73.86656564973001 40.731773141489)
7,40372618,BURGER KING,Queens,15405,ROCKAWAY BOULEVARD,11434.0,7185275044,Hamburgers,11/30/2018,Violations were cited in the following area(s).,...,11/30/2018,05/16/2020,Cycle Inspection / Re-inspection,412.0,31.0,29400.0,4266764.0,4.123100e+09,QN02,POINT (-73.78098427964301 40.667935318774)
8,40388016,WENDY'S,Bronx,2140,WESTCHESTER AVENUE,10462.0,7184096174,Hamburgers,12/23/2019,Violations were cited in the following area(s).,...,12/23/2019,05/16/2020,Cycle Inspection / Initial Inspection,209.0,18.0,9200.0,2088065.0,2.038140e+09,BX59,POINT (-73.85314065129799 40.834031846299)
9,40393742,POPEYES/DUNKIN DONUTS,Manhattan,321,WEST 125 STREET,10027.0,2129329636,Chicken,03/12/2020,Violations were cited in the following area(s).,...,03/12/2020,05/16/2020,Cycle Inspection / Initial Inspection,110.0,9.0,25700.0,1059310.0,1.019520e+09,MN11,POINT (-73.951791635813 40.810436705074)


In [112]:
df_FastFood.shape

(435, 25)

Spatial join fast food locations to CD polygons

In [113]:
df_sjoin = gpd.sjoin(df_cd, df_FastFood, how = 'inner', op = 'intersects')
df_sjoin

,boro_cd,shape_area,shape_leng,GEO_ID,GEO_LABEL,GEO_DISPLAY_NAME,total_population_cd,obesity_cd,geometry,index_right,...,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,210,...,A,09/28/2018,05/16/2020,Cycle Inspection / Re-inspection,101.0,1.0,900.0,1085792.0,1.000020e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,21,...,A,09/02/2016,05/16/2020,Cycle Inspection / Re-inspection,101.0,1.0,900.0,1000030.0,1.000110e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,392,...,A,12/06/2019,05/16/2020,Pre-permit (Operational) / Initial Inspection,101.0,1.0,1502.0,1090416.0,1.000760e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,281,...,A,04/03/2019,05/16/2020,Cycle Inspection / Initial Inspection,101.0,1.0,1502.0,1001195.0,1.000780e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,233,...,None,None,05/16/2020,Cycle Inspection / Initial Inspection,101.0,1.0,1300.0,1001043.0,1.000520e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,182,...,A,12/17/2018,05/16/2020,Cycle Inspection / Initial Inspection,101.0,1.0,1502.0,1001212.0,1.000790e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,285,...,None,None,05/16/2020,Calorie Posting / Initial Inspection,101.0,1.0,1502.0,1001223.0,1.000790e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,314,...,A,04/08/2019,05/16/2020,Pre-permit (Operational) / Initial Inspection,101.0,1.0,1502.0,1089384.0,1.000790e+09,MN25
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,11,...,A,12/11/2017,05/16/2020,Cycle Inspection / Re-inspection,101.0,1.0,3300.0,1001642.0,1.001500e+09,MN24
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,401,...,N,None,05/16/2020,Pre-permit (Operational) / Initial Inspection,101.0,1.0,3300.0,1079211.0,1.001520e+09,MN24


In [114]:
df_sjoin.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 426 entries, 0 to 58
Data columns (total 34 columns):
boro_cd                  426 non-null float64
shape_area               426 non-null float64
shape_leng               426 non-null float64
GEO_ID                   426 non-null float64
GEO_LABEL                426 non-null object
GEO_DISPLAY_NAME         426 non-null object
total_population_cd      426 non-null float64
obesity_cd               426 non-null int64
geometry                 426 non-null object
index_right              426 non-null int64
CAMIS                    426 non-null int64
DBA                      426 non-null object
BORO                     426 non-null object
BUILDING                 426 non-null object
STREET                   426 non-null object
ZIPCODE                  426 non-null float64
PHONE                    426 non-null object
CUISINE DESCRIPTION      426 non-null object
INSPECTION DATE          426 non-null object
ACTION                   413 n

426/435 fast food establishments were in the community districts

In [115]:
# see how many boro_cds had fast food establishments
df_sjoin['boro_cd'].nunique()

58

Appears that 58/59 of the community districts have fast food establishments within them

Count the number of fast food establishments per community district

In [116]:
cd_counts = df_sjoin.pivot_table(index = ['boro_cd'],
                                aggfunc = {'CAMIS': 'nunique'}).reset_index().sort_values('CAMIS', ascending = False)
cd_counts

,boro_cd,CAMIS
4,105.0,19
52,412.0,19
56,502.0,15
40,318.0,13
10,111.0,13
23,212.0,13
28,305.0,11
11,112.0,11
0,101.0,11
43,403.0,11


In [117]:
cd_counts['CAMIS'].describe()

count    58.000000
mean      7.344828
std       3.730160
min       1.000000
25%       5.000000
50%       6.500000
75%       9.750000
max      19.000000
Name: CAMIS, dtype: float64

Merge back to origin cd geometries

In [118]:
df_FastFood_cd = df_cd.merge(cd_counts, how = 'left',
                            on = 'boro_cd').rename(columns = {'CAMIS':'number_of_fast_food'})
df_FastFood_cd

,boro_cd,shape_area,shape_leng,GEO_ID,GEO_LABEL,GEO_DISPLAY_NAME,total_population_cd,obesity_cd,geometry,number_of_fast_food
0,101.0,4.268659e+07,73762.393219,101.0,Manhattan CD 001,Financial District,130217.345712,24,(POLYGON ((-74.0438776163991 40.69018767537123...,11.0
1,102.0,3.768921e+07,34130.595861,102.0,Manhattan CD 002,Greenwich Village and Soho,101000.735022,26,"POLYGON ((-73.9968399392649 40.7373608893982, ...",5.0
2,103.0,4.687970e+07,30468.340600,103.0,Manhattan CD 003,Lower East Side and Chinatown,103831.154791,32,"POLYGON ((-73.98877931684083 40.7339653992359,...",6.0
3,104.0,4.931179e+07,67623.946684,104.0,Manhattan CD 004,Clinton and Chelsea,126139.483095,24,POLYGON ((-73.99393694248319 40.77317951096692...,6.0
4,105.0,4.379030e+07,35288.305200,105.0,Manhattan CD 005,Midtown,193083.299602,21,POLYGON ((-73.97301487176121 40.76427887944521...,19.0
5,106.0,3.872909e+07,42705.937484,106.0,Manhattan CD 006,Stuyvesant Town and Turtle Bay,193731.204933,15,(POLYGON ((-73.96128450788402 40.7301590617016...,7.0
6,107.0,5.315282e+07,39863.701384,107.0,Manhattan CD 007,Upper West Side,103174.978364,28,POLYGON ((-73.95964685423152 40.80115642325645...,4.0
7,108.0,5.516880e+07,53561.096900,108.0,Manhattan CD 008,Upper East Side,164500.003324,28,(POLYGON ((-73.94180032729426 40.7690469266246...,5.0
8,109.0,4.189218e+07,34959.184938,109.0,Manhattan CD 009,Morningside Heights and Hamilton Heights,166931.655361,26,POLYGON ((-73.94014131210091 40.83037198888071...,3.0
9,110.0,3.908463e+07,35825.290700,110.0,Manhattan CD 010,Central Harlem,88571.946738,41,POLYGON ((-73.93445239397371 40.83598096014986...,5.0


In [122]:
import mapclassify
import geoplot

# let's split up the gym counts by quantiles ranges
count = df_FastFood_cd['number_of_fast_food']
scheme = mapclassify.Quantiles(count, k=5)

geoplot.choropleth(
    df_FastFood_cd, hue=count, scheme=scheme,
    cmap='Blues', figsize=(12, 6)
);

ModuleNotFoundError: No module named 'geoplot'

Export as geojson 

In [121]:
df_FastFood_cd.to_file('FastFood_cd.geojson', driver = 'GeoJSON', encoding = 'utf-8')